<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/Hier_TransformerI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Uncomment if running on googlecolab
!pip install hickle
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/PerCom2021-FL-master/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 1.3 MB/s eta 0:00:00
Mounted at /content/drive/
[Errno 2] No such file or directory: 'drive/MyDrive/PerCom2021-FL-master/'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/PerCom2021-FL-master/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/PerCom2021-FL-master/'
/content


In [96]:
import numpy as np
import os
import pandas as pd
from subprocess import call
import requests
np.random.seed(0)
import urllib.request
import zipfile
import hickle as hkl
import tensorflow as tf

In [97]:
# functions for loading and downloading the dataset

# load a single file as a numpy array
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	filenames = list()
	# body acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# Framing data by windows
def segmentData(accData,time_step,step):
    segmentAccData = list()
    for i in range(0, accData.shape[0] - time_step,step):
        segmentAccData.append(accData[i:i+time_step,:])
    return segmentAccData

# download function for datasets
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [98]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

In [128]:
def splitData(xtrain,ytrain,xtest,ytest,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==0) | (ytrain==3)]
    ytrain = ytrain[(ytrain==0) | (ytrain==3)]
    xtest = xtest[(ytest==0) | (ytest==3)]
    ytest = ytest[(ytest==0) |(ytest==3)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==0) | (ytrain==1)|(ytrain==2)]
    ytrain = ytrain[(ytrain==0) | (ytrain==1)|(ytrain==2)]
    xtest = xtest[(ytest==0) | (ytest==1)|(ytest==2)]
    ytest = ytest[(ytest==0) | (ytest==1)|(ytest==2)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==3) | (ytrain==4)|(ytrain==5)]
    ytrain = ytrain[(ytrain==3) | (ytrain==4)|(ytrain==5)]
    xtest = xtest[(ytest==3) | (ytest==4)|(ytest==5)]
    ytest = ytest[(ytest==3) | (ytest==4)|(ytest==5)]
 # elif(type=='transition'):
 #   xtrain = xtrain[(trainlabels==7) | (trainlabels==8)|(trainlabels==9)|(trainlabels==10)|(trainlabels==11)|(trainlabels==12)]
 #   ytrain = ytrain[(trainlabels==7) | (trainlabels==8)|(trainlabels==9)|(trainlabels==10)|(trainlabels==11)|(trainlabels==12)]
 # elif(type=='primary'):
  #  xtrain = xtrain[(trainlabels==1) | (trainlabels==2)|(trainlabels==3)|(trainlabels==4)|(trainlabels==5)|(trainlabels==6)]
  #  ytrain = ytrain[(trainlabels==1) | (trainlabels==2)|(trainlabels==3)|(trainlabels==4)|(trainlabels==5)|(trainlabels==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

#  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest

  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels

In [100]:
# download and unzipping dataset
os.makedirs('dataset',exist_ok=True)
print("downloading...")
data_directory = os.path.abspath("dataset/UCI HAR Dataset.zip")
if not os.path.exists(data_directory):
    download_url("https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.zip",data_directory)
    print("download done")
else:
    print("dataset already downloaded")

data_directory2 = os.path.abspath("dataset/UCI HAR Dataset")
if not os.path.exists(data_directory2):
    print("extracting data")
    with zipfile.ZipFile(data_directory, 'r') as zip_ref:
        zip_ref.extractall(os.path.abspath("dataset/"))
    print("data extracted in " + data_directory2)
else:
    print("Data already extracted in " + data_directory2)

downloading...
dataset already downloaded
Data already extracted in /content/dataset/UCI HAR Dataset


In [101]:
trainSubjectList = pd.read_csv('dataset/UCI HAR Dataset/train/subject_train.txt', header=None, delim_whitespace=True).values
testSubjectList = pd.read_csv('dataset/UCI HAR Dataset/test/subject_test.txt', header=None, delim_whitespace=True).values

In [102]:
# load all train
trainX, trainy = load_dataset('train', 'dataset/UCI HAR Dataset/')
trainy = np.asarray([x - 1 for x in trainy])

# load all test
testX, testy = load_dataset('test', 'dataset/UCI HAR Dataset/')
testy = np.asarray([x - 1 for x in testy])

In [103]:
combinedData = np.vstack((trainX,testX))

In [104]:
subjectList = np.vstack((trainSubjectList,testSubjectList)).squeeze()
labels = np.vstack((trainy,testy)).squeeze()

In [105]:
nbOfSubjects = len(np.unique(subjectList))

In [106]:
subjectDataDict = {new_list: [] for new_list in range(nbOfSubjects)}
subjectLabelDict = {new_list: [] for new_list in range(nbOfSubjects)}

In [107]:
combinedData.shape

(10299, 128, 6)

In [108]:
meanAcc = np.mean(combinedData[:,:,:3])
stdAcc = np.std(combinedData[:,:,:3])
varAcc = np.var(combinedData[:,:,:3])

meanGyro = np.mean(combinedData[:,:,3:])
stdGyro = np.std(combinedData[:,:,3:])
varGyro = np.var(combinedData[:,:,3:])

normalizedAllAcc = (combinedData[:,:,:3] - meanAcc) / stdAcc
normalizedAllGyro = (combinedData[:,:,3:] - meanGyro) / stdGyro
normalizedAll = np.dstack((normalizedAllAcc,normalizedAllGyro))

In [109]:
trainy = np.squeeze(trainy)
testy = np.squeeze(testy)

In [110]:
dataName = 'UCI'
os.makedirs('datasetStandardized/'+dataName, exist_ok=True)
hkl.dump(normalizedAll[:trainX.shape[0]],'datasetStandardized/'+dataName+ '/trainX.hkl' )
hkl.dump(normalizedAll[trainX.shape[0]:],'datasetStandardized/'+dataName+ '/testX.hkl' )
hkl.dump(trainy,'datasetStandardized/'+dataName+ '/trainY.hkl' )
hkl.dump(testy,'datasetStandardized/'+dataName+ '/testY.hkl' )

In [111]:
trainX.shape

(7352, 128, 6)

In [19]:
!pip install tensorflow==2.10.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 53.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0

In [20]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%ls

dataset/  datasetStandardized/  drive/  sample_data/


In [21]:
!pip install keras_flops

  Using cached keras_flops-0.1.2-py3-none-any.whl (5.3 kB)


In [112]:
import os
randomSeed = 1
os.environ['PYTHONHASHSEED']=str(randomSeed)
import numpy as np
import tensorflow as tf
import csv
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import time
import hickle as hkl
import random
import math
import logging
import shutil
import gc
import sys
import sklearn.manifold
import seaborn as sns
import argparse
import matplotlib.gridspec as gridspec
import __main__ as main
import model
import utils
from keras_flops import get_flops

In [113]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [114]:
# which CPU/GPU to use
# "-1,0,1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set The Default Hyperparameters Here

architecture = "HART"
# MobileHART, HART

# RealWorld,HHAR,UCI,SHL,MotionSense, COMBINED
dataSetName = 'UCI'

#BALANCED, UNBALANCED
dataConfig = "BALANCED"

# Show training verbose: 0,1
showTrainVerbose = 1

# input window size
segment_size = 128

# input channel count
num_input_channels = 6

learningRate = 3e-4

# model drop out rate
dropout_rate = 0.3

# local epoch
localEpoch = 200
# or 4
frameLength = 16

timeStep = 16

positionDevice = ''
# ['chest','forearm','head','shin','thigh','upperarm','waist']
# ['nexus4', 'lgwatch','s3', 's3mini','gear','samsungold']
tokenBased = False

measureEnergy = False

In [115]:
# hyperparameter for the model

batch_size = 256
projection_dim = 192
filterAttentionHead = 4

# To adjust the number of blocks in for HART, Add or remove the conv kernel size here
# each conv kernel length is for one HART block
convKernels = [3, 7, 15, 31, 31, 31]

In [116]:
def add_fit_args(parser):
    """
    parser : argparse.ArgumentParser
    return a parser added with args required by fit
    """
    # Training settings
    parser.add_argument('--batch_size', type=int, default=batch_size,
                        help='Batch size of the training')
    parser.add_argument('--localEpoch', type=int, default=localEpoch,
                        help='Number of epochs for training')
    parser.add_argument('--architecture', type=str, default=architecture,
                        help='Choose between HART or MobileHART')
    parser.add_argument('--projection_dim', type=int, default=projection_dim,
                        help='Size of the projection dimensions')
    parser.add_argument('--frame_length', type=int, default=frameLength,
                help='Patch Size')
    parser.add_argument('--time_step', type=int, default=timeStep,
            help='Stride Size')
    parser.add_argument('--dataset', type=str, default=dataSetName,
        help='Dataset')
    parser.add_argument('--tokenBased', type=bool, default=tokenBased,
        help='Use Token or Global Average Pooling')
    parser.add_argument('--positionDevice', type=str, default=positionDevice,
        help='Test is done other position not in training, if empty, uses a 70/10/20 train/dev/test ratio ')
    args = parser.parse_args()
    return args

In [117]:
def is_interactive():
    return not hasattr(main, '__file__')

In [118]:
if not is_interactive():
    args = add_fit_args(argparse.ArgumentParser(description='Human Activity Recognition Transformer'))
    localEpoch = args.localEpoch
    batch_size = args.batch_size
    architecture = args.architecture
    projection_dim = args.projection_dim
    frameLength = args.frame_length
    timeStep = args.time_step
    dataSetName = args.dataset
    tokenBased = args.tokenBased
    positionDevice = args.positionDevice

input_shape = (segment_size,num_input_channels)
projectionHalf = projection_dim//2
projectionQuarter = projection_dim//4

transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers

R = projectionHalf // filterAttentionHead
assert R * filterAttentionHead == projectionHalf


segmentTime = [x for x in range(0,segment_size - frameLength + timeStep,timeStep)]
assert R * filterAttentionHead == projectionHalf
if(positionDevice != ''):
    assert dataSetName == "RealWorld" or dataSetName == "HHAR"

In [119]:
# specifying activities and where the results will be stored
if(dataSetName == 'UCI'):
    ACTIVITY_LABEL = ['Walking', 'Upstair','Downstair', 'Sitting', 'Standing', 'Lying']
elif(dataSetName == "RealWorld"):
    ACTIVITY_LABEL = ['Downstairs','Upstairs', 'Jumping','Lying', 'Running', 'Sitting', 'Standing', 'Walking']
elif(dataSetName == "Motion_Sense"):
    ACTIVITY_LABEL = ['Downstairs', 'Upstairs', 'Sitting', 'Standing', 'Walking', 'Jogging']
elif(dataSetName == "HHAR"):
    ACTIVITY_LABEL = ['Sitting', 'Standing', 'Walking', 'Upstair', 'Downstairs', 'Biking']
else:
#     SHL
    ACTIVITY_LABEL = ['Standing','Walking','Runing','Biking','Car','Bus','Train','Subway']

activityCount = len(ACTIVITY_LABEL)

architectureType = str(architecture)+'_'+str(int(frameLength))+'frameLength_'+str(timeStep)+'TimeStep_'+str(projection_dim)+"ProjectionSize_"+str(learningRate)+'LR'
if(tokenBased):
    architectureType = architectureType + "_tokenBased"

if(positionDevice != ''):
    architectureType = architectureType + "_PositionWise_" + str(positionDevice)
mainDir = './'

if(localEpoch < 20):
    architectureType =  "Tests/"+str(architectureType)
filepath = mainDir +'HART_Results/'+architectureType+'/'+dataSetName+'/'
os.makedirs(filepath, exist_ok=True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

attentionPath = filepath+"attentionImages/"
os.makedirs(attentionPath, exist_ok=True)

bestModelPath = filepath + 'bestModels/'
os.makedirs(bestModelPath, exist_ok=True)

currentModelPath = filepath + 'currentModels/'
os.makedirs(currentModelPath, exist_ok=True)


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
np.random.seed(randomSeed)
tf.keras.utils.set_random_seed(randomSeed)
tf.random.set_seed(randomSeed)
random.seed(randomSeed)


Num GPUs Available:  1


In [121]:
if(dataSetName == "COMBINED"):
    datasetList = ["UCI","RealWorld","HHAR", "Motion_Sense","SHL_128"]
    ACTIVITY_LABEL = ['Walk', 'Upstair', 'Downstair', 'Sit', 'Stand', 'Lay', 'Jump','Run', 'Bike', 'Car', 'Bus', 'Train', 'Subway']
    activityCount = len(ACTIVITY_LABEL)
    UCI = [0,1,2,3,4,5]
    REALWORLD_CLIENT = [2,1,6,5,7,3,4,0]
    HHAR = [3,4,0,1,2,8]
    Motion_Sense = [2,1,3,4,0,7]
    SHL = [4,0,7,8,9,10,11,12]

    centralTrainData = []
    centralTrainLabel = []
    centralTestData = []
    centralTestLabel = []
    for dataSetName in datasetList:
        clientCount = utils.returnClientByDataset(dataSetName)
        loadedDataset = utils.loadDataset(dataSetName,clientCount,dataConfig,randomSeed,mainDir+'datasets/')
        centralTrainData.append(loadedDataset.centralTrainData)
        centralTrainLabel.append(loadedDataset.centralTrainLabel)
        centralTestData.append(loadedDataset.centralTestData)
        centralTestLabel.append(loadedDataset.centralTestLabel)
        print(dataSetName + " has class :" +str(np.unique(centralTrainLabel[-1])))
        del loadedDataset

    centralTestLabelAligned = []
    centralTrainLabelAligned = []
    combinedAlignedData = centralTestData
    for index, datasetName in enumerate(datasetList):
        if(datasetName == 'UCI'):
            centralTrainLabelAligned.append(centralTrainLabel[index])
            centralTestLabelAligned.append(centralTestLabel[index])
        elif(datasetName == 'RealWorld'):
            centralTrainLabelAligned.append(np.hstack([REALWORLD_CLIENT[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([REALWORLD_CLIENT[labelIndex] for labelIndex in centralTestLabel[index]]))

        elif(datasetName == 'HHAR'):
            centralTrainLabelAligned.append(np.hstack([HHAR[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([HHAR[labelIndex] for labelIndex in centralTestLabel[index]]))
        elif(datasetName == 'Motion_Sense'):
            centralTrainLabelAligned.append(np.hstack([Motion_Sense[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([Motion_Sense[labelIndex] for labelIndex in centralTestLabel[index]]))
        else:
            centralTrainLabelAligned.append(np.hstack([SHL[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([SHL[labelIndex] for labelIndex in centralTestLabel[index]]))
    centralTrainData = np.vstack((centralTrainData))
    centralTestData = np.vstack((centralTestData))
    centralTrainLabel = np.hstack((centralTrainLabelAligned))
    centralTestLabel = np.hstack((centralTestLabelAligned))
else:
    clientCount = utils.returnClientByDataset(dataSetName)
    datasetLoader = utils.loadDataset(dataSetName,clientCount,dataConfig,randomSeed,mainDir+'datasets/')
    centralTrainData = datasetLoader.centralTrainData
    centralTrainLabel = datasetLoader.centralTrainLabel

    centralTestData = datasetLoader.centralTestData
    centralTestLabel = datasetLoader.centralTestLabel

    clientOrientationTrain = datasetLoader.clientOrientationTrain
    clientOrientationTest = datasetLoader.clientOrientationTest
    orientationsNames = datasetLoader.orientationsNames

In [122]:
# If working on RealWorld or HHAR with specified position/device, we remove one and use it as the test set and combine the others for training
if(positionDevice != '' or dataSetName == 'UCI'):
    if(dataSetName == "RealWorld"):
        totalData = np.vstack((centralTrainData,centralTestData))
        totalLabel = np.hstack((centralTrainLabel,centralTestLabel))
        totalOrientation = np.hstack((np.hstack((clientOrientationTrain)), np.hstack((clientOrientationTest))))
        totalIndex = list(range(totalOrientation.shape[0]))
        testDataIndex = np.where(totalOrientation == orientationsNames.index(positionDevice))[0]
        trainDataIndex = np.delete(totalIndex,testDataIndex)

        centralTrainData = totalData[trainDataIndex]
        centralTestData = totalData[testDataIndex]

        centralTrainLabel = totalLabel[trainDataIndex]
        centralTestLabel = totalLabel[testDataIndex]
    elif(dataSetName == "HHAR"):
        totalData = np.vstack((centralTrainData,centralTestData))
        totalLabel = np.hstack((centralTrainLabel,centralTestLabel))
        totalOrientation = np.hstack((np.hstack((clientOrientationTrain)), np.hstack((clientOrientationTest))))
        totalIndex = list(range(totalOrientation.shape[0]))
        # 0 is for nexus
        testDataIndex = np.where(totalOrientation == orientationsNames.index(positionDevice))[0]
        trainDataIndex = np.delete(totalIndex,testDataIndex)

        centralTrainData = totalData[trainDataIndex]
        centralTestData = totalData[testDataIndex]

        centralTrainLabel = totalLabel[trainDataIndex]
        centralTestLabel = totalLabel[testDataIndex]
#         when using positions for evalaution, there is no test set , dev=test is the same
    centralDevData = centralTestData
    centralDevLabel = centralTestLabel
else:
#     using a 70 10 20 ratio
    centralTrainData, centralDevData, centralTrainLabel, centralDevLabel = train_test_split(centralTrainData, centralTrainLabel, test_size=0.125, random_state=randomSeed)

In [123]:
# Compute class weight
temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(centralTrainLabel),
                                                 y = centralTrainLabel.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

In [154]:
# One Hot of labels
centralTrainLabel = tf.one_hot(
    centralTrainLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)
centralTestLabel = tf.one_hot(
    centralTestLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)
centralDevLabel = tf.one_hot(
    centralDevLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)

In [129]:
centralTestLabel

array([4, 4, 4, ..., 1, 1, 1])

In [150]:
optimizer = tf.keras.optimizers.Adam(learningRate)
optimizer0 = tf.keras.optimizers.Adam(learningRate)
optimizer2 = tf.keras.optimizers.Adam(learningRate)
optimizer3 = tf.keras.optimizers.Adam(learningRate)


if(architecture == "HART"):
    linear_classifier = model.HART(input_shape,6)

    model_classifier = model.HART(input_shape,2)
    dynamic_classifier = model.HART(input_shape,3)
    static_classifier = model.HART(input_shape,3)
else:
    model_classifier = model.mobileHART_XS(input_shape,activityCount, convKernels = convKernels)

linear_classifier.compile(
    optimizer=optimizer0,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
model_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
dynamic_classifier.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
static_classifier.compile(
    optimizer=optimizer3,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
#model_classifier.summary()

In [151]:
centralTrainData.shape,centralTrainLabel.shape, trainy.shape

((7352, 128, 6), (7352,), (7352,))

In [127]:
np.unique(trainy)

array([0, 1, 2, 3, 4, 5])

In [ ]:
tee= centralTrainData[(trainy==1)|(trainy==2)|(trainy==3)]

In [ ]:
wee= centralTrainLabel[(trainy==1)|(trainy==2)|(trainy==3)]

In [ ]:
tee.shape, wee.shape

((3345, 128, 6), TensorShape([3345, 6]))

In [40]:
trainy, testy

(array([4, 4, 4, ..., 1, 1, 1]), array([4, 4, 4, ..., 1, 1, 1]))

In [ ]:
#splitData(xtrain,ytrain,xtest,ytest,type):

In [72]:
centralTestData.shape,testy.shape

((2947, 128, 6), (2947,))

**Split data**

In [155]:
masterxtrain,masterxtest,masterytrain,masterytest,masterlabels=splitData(centralTrainData,trainy,centralTestData,testy,"master")   #return xtrain,xtest,ytrain,ytest,testlabels
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels=splitData(centralTrainData,trainy,centralTestData,testy,"dynamic")
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels=splitData(centralTrainData,trainy,centralTestData,testy,"static")

In [156]:
masterxtrain.shape,masterytrain.shape,masterxtest.shape,masterytest.shape

((2512, 128, 6), (2512, 2), (987, 128, 6), (987, 2))

In [136]:
staticytrain, staticlabels

(array([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]),
 array([4, 4, 4, ..., 5, 5, 5]))

**training**

In [157]:
localEpoch=100

checkpoint_filepath = filepath+"bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

start_time = time.time()
history = model_classifier.fit(
    x=masterxtrain,
    y=masterytrain,
#    validation_data = (centralDevData,centralDevLabel),
    batch_size=batch_size,
    epochs=localEpoch,
    verbose=showTrainVerbose,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)

history2 = dynamic_classifier.fit(
    x=dynamicxtrain,
    y=dynamicytrain,
 #   validation_data = (centralDevData,centralDevLabel),
    batch_size=batch_size,
    epochs=localEpoch,
    verbose=showTrainVerbose,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)
history3 = static_classifier.fit(
    x=staticxtrain,
    y=staticytrain,
 #   validation_data = (centralDevData,centralDevLabel),
    batch_size=batch_size,
    epochs=localEpoch,
    verbose=showTrainVerbose,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)
end_time = time.time() - start_time

#model_classifier.save_weights(filepath + 'bestTrain.h5')
#model_classifier.load_weights(checkpoint_filepath)
#_, accuracy = model_classifier.evaluate(centralTestData, centralTestLabel)
#print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Epoch 1/100


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x7c6936006dd0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x7c6936006dd0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 9/10 [==========================>...] - ETA: 0s - loss: 0.4835 - accuracy: 0.8273

10/10 [==============================] - 55s 57ms/step - loss: 0.4682 - accuracy: 0.8404
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 0.2927 - accuracy: 0.9777

10/10 [==============================] - 0s 48ms/step - loss: 0.2927 - accuracy: 0.9777
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 0.2556 - accuracy: 0.9896

...

16/16 [==============================] - 1s 78ms/step - loss: 0.3524 - accuracy: 0.9904
Epoch 97/100
16/16 [==============================] - ETA: 0s - loss: 0.3507 - accuracy: 0.9911

16/16 [==============================] - 1s 85ms/step - loss: 0.3507 - accuracy: 0.9911
Epoch 98/100
16/16 [==============================] - ETA: 0s - loss: 0.3499 - accuracy: 0.9914

16/16 [==============================] - 1s 80ms/step - loss: 0.3499 - accuracy: 0.9914
Epoch 99/100
15/16 [===========================>..] - ETA: 0s - loss: 0.3516 - accuracy: 0.9904

16/16 [==============================] - 1s 53ms/step - loss: 0.3514 - accuracy: 0.9904
Epoch 100/100
16/16 [==============================] - ETA: 0s - loss: 0.3522 - accuracy: 0.9894

16/16 [==============================] - 1s 49ms/step - loss: 0.3522 - accuracy: 0.9894


In [160]:
history0 = linear_classifier.fit(
    x=centralTrainData,
    y=centralTrainLabel,
#    validation_data = (centralDevData,centralDevLabel),
    batch_size=batch_size,
    epochs=localEpoch,
    verbose=showTrainVerbose,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)

Epoch 1/100


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x7c69e0852440>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x7c69e0852440>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
29/29 [==============================] - ETA: 0s - loss: 1.2022 - accuracy: 0.5989

29/29 [==============================] - 44s 71ms/step - loss: 1.2022 - accuracy: 0.5989
Epoch 2/100
29/29 [==============================] - ETA: 0s - loss: 0.8818 - accuracy: 0.7865

29/29 [==============================] - 1s 49ms/step - loss: 0.8818 - accuracy: 0.7865
Epoch 3/100
29/29 [==============================] - ETA: 0s - loss: 0.7492 - accuracy: 0.8560

...

29/29 [==============================] - 1s 51ms/step - loss: 0.4416 - accuracy: 0.9898
Epoch 97/100
28/29 [===========================>..] - ETA: 0s - loss: 0.4383 - accuracy: 0.9908

29/29 [==============================] - 2s 52ms/step - loss: 0.4382 - accuracy: 0.9910
Epoch 98/100
28/29 [===========================>..] - ETA: 0s - loss: 0.4402 - accuracy: 0.9893

29/29 [==============================] - 1s 49ms/step - loss: 0.4402 - accuracy: 0.9891
Epoch 99/100
29/29 [==============================] - ETA: 0s - loss: 0.4343 - accuracy: 0.9931

29/29 [==============================] - 2s 74ms/step - loss: 0.4343 - accuracy: 0.9931
Epoch 100/100
29/29 [==============================] - ETA: 0s - loss: 0.4367 - accuracy: 0.9920

29/29 [==============================] - 2s 77ms/step - loss: 0.4367 - accuracy: 0.9920


In [161]:
print("time is "+str(end_time))


time is 422.10405707359314


In [162]:
_, accuracy = model_classifier.evaluate(masterxtest, masterytest)
print(f"Test accuracy(master): {round(accuracy * 100, 2)}%")
_, accuracy = dynamic_classifier.evaluate(dynamicxtest, dynamicytest)
print(f"Test accuracy(dynamic): {round(accuracy * 100, 2)}%")
_, accuracy = static_classifier.evaluate(staticxtest, staticytest)
print(f"Test accuracy(static): {round(accuracy * 100, 2)}%")
_, accuracy = linear_classifier.evaluate(centralTestData, centralTestLabel)
print(f"Test accuracy(linear): {round(accuracy * 100, 2)}%")

Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69e47600d0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69e47600d0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
31/31 [==============================] - 2s 13ms/step - loss: 0.2613 - accuracy: 0.9757
Test accuracy(master): 97.57%


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69dfe71510>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69dfe71510>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
44/44 [==============================] - 2s 14ms/step - loss: 0.4829 - accuracy: 0.9293


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c6936680dc0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Test accuracy(dynamic): 92.93%
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c6936680dc0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
49/49 [==============================] - 3s 22ms/step - loss: 0.5054 - accuracy: 0.9173
Test accuracy(static): 91.73%


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69dfdf8550>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x7c69dfdf8550>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 3s 13ms/step - loss: 0.6446 - accuracy: 0.9247
Test accuracy(linear): 92.47%


In [35]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1


In [163]:
yexpect = np.concatenate([dynamiclabels,staticlabels], axis=0)

yexpect.shape

(2947,)

In [164]:
merge_arr = np.concatenate([dynamicxtest, staticxtest], axis=0)
merge_arr.shape

(2947, 128, 6)

In [165]:
pre=linear_classifier.predict(merge_arr)
ypred=np.argmax(pre,axis=1)

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7c69dfbbfbe0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7c69dfbbfbe0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 3s 12ms/step


In [166]:
ypred

array([0, 0, 0, ..., 5, 5, 5])

In [167]:
scores=[]
dynamiccount=0
staticcount=0

dynamics = np.array([0,1,2])
statics = np.array([3,4,5])

for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,6)
    if(ypred[i] in [0,1,2]):
        predict_x=dynamic_classifier.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(ypred[i] in [3,4,5]):
        predict_x=static_classifier.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


    print(i)


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7c69e4875e10>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7c69e4875e10>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 1s/step
0
1/1 [==============================] - 0s 42ms/step
1
1/1 [==============================] - 0s 32ms/step
2
1/1 [==============================] - 0s 33ms/step
3
1/1 [==============================] - 0s 30ms/step
4
1/1 [==============================] - 0s 31ms/step
5
1/1 [==============================] - 0s 33ms/step
6
1/1 [==============================] - 0s 30ms/step
7
1/1 [===

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7c69e0d2be20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 50ms/step
447
1/1 [==============================] - 0s 47ms/step
448
1/1 [==============================] - 0s 47ms/step
449
1/1 [==============================] - 0s 52ms/step
450
1/1 [==============================] - 0s 50ms/step
451
1/1 [==============================] - 0s 49ms/step
452
1/1 [==============================] - 0s 49ms/step
453
1/1 [==============================] - 0s 46ms/step
454
1/1 [==============================] - 0s 45ms/step
455
1/1 [==============================] - 0s 51ms/step
456
1/1 [==============================] - 0s 51ms/step
457
1/1 [==============================] - 0s 40ms/step
458
1/1 [==============================] - 0s 29ms/step
459
1/1 [==============================] - 0s 34ms/step
460
1/1 [==============================] - 0s 32ms/step
461
1/1 [==============================] - 0s 31ms/step
462
1/1 [==============================] - 0s 30ms/step
4

In [168]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))

Accuracy score TEST: 92.331
